In [161]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import numpy as np
import pandas as pd

# Generation

In [65]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

prompt = "It is important for all countries to try harder to reduce carbon emissions because"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# sample up to 30 tokens
torch.manual_seed(11)
list_out=[]
for i in range (1):
    for j in range (8):
        outputs = model.generate(input_ids, do_sample=True, max_length=30, temperature =11.6+0.75*i,top_p=(j+1)/30,repetition_penalty=1.1)
        list_out.append(outputs)

for i in range (1):
    for j in range (8):
        print("temperature =",11.6+0.75*i,"top_p=",(j+1)/30)
        print(tokenizer.batch_decode(list_out[i*4+j], skip_special_tokens=True))
        

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

KeyboardInterrupt: 

# Show Probability

In [172]:
outputs = model.generate(input_ids, do_sample=False, num_beams=1, max_length=30, temperature =1,top_p=0.99,return_dict_in_generate=True, output_scores=True,repetition_penalty=1.0)
print(tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True))
transition_scores = model.compute_transition_scores(outputs.sequences, outputs.scores, normalize_logits=True)

input_length = 1 if model.config.is_encoder_decoder else input_ids.shape[1] #?
generated_tokens = outputs.sequences[:, input_length:]

df=pd.DataFrame(columns=["token","string","probability(%)"])

i=0
for score in transition_scores[0]:
    tok=generated_tokens[0,i]
    # | token | token string | logits | probability
    #print(f"| {tok:5d} {str(tokenizer.batch_decode(generated_tokens[:,i], skip_special_tokens=True)):12s} {score.numpy():.3f} | {np.exp(score.numpy()):.2%}")
    df.loc[i] = [tok, tokenizer.decode(generated_tokens[:,i]), 100*np.exp(score.numpy())]
    i+=1
    
df

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['It is important for all countries to try harder to reduce carbon emissions because it is a key part of the global economy.\n\n"We need to']


,token,string,probability(%)
0,tensor(340),it,16.150859
1,tensor(318),is,33.391905
2,tensor(257),a,15.054569
3,tensor(1994),key,6.984391
4,tensor(636),part,15.045089
5,tensor(286),of,98.910081
6,tensor(262),the,26.422513
7,tensor(3298),global,11.689099
8,tensor(3773),economy,19.673710
9,tensor(13),.,33.241472


# Generate probability tree

In [220]:
from treelib import Node, Tree

prompt = "It is important for all countries to try harder to reduce carbon emissions because"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# sample up to 30 tokens
torch.manual_seed(11)
outputs = model.generate(input_ids, do_sample=False, num_beams=3, max_length=30, temperature =1,top_p=0.99,return_dict_in_generate=True, output_scores=True,repetition_penalty=1.0)
sequences = outputs.sequences # sequences is a tensor of shape (batch_size, sequence_length)
scores = outputs.scores
print("sequences=",type(sequences),len(sequences[0]))
print(tokenizer.batch_decode(sequences, skip_special_tokens=True))



tree = Tree()
node=0
tree.create_node(str(tokenizer.batch_decode(sequences[0:1,13], skip_special_tokens=True)), node)  # root node
parent=node

i=0
for score in scores:
    token_num=sequences[0,i+14]
     
    
    tok=torch.argmax(score[0])
    prob=np.round(100*np.exp(max(score[0]).numpy()))
    string=str([tokenizer.decode(tok)])+" "+str(prob)+"%"
    score[0][torch.argmax(score[0])]-=100
    node+=1
    if tok==token_num:
        parent_next=node
    tree.create_node(string, node ,parent=parent)
    
    
    tok=torch.argmax(score[0])
    prob=np.round(100*np.exp(max(score[0]).numpy()))
    string=str([tokenizer.decode(tok)])+" "+str(prob)+"%"
    score[0][torch.argmax(score[0])]-=100
    node+=1
    if tok==token_num:
        parent_next=node
    tree.create_node(string, node ,parent=parent)

    tok=torch.argmax(score[0])
    prob=np.round(100*np.exp(max(score[0]).numpy()))
    string=str([tokenizer.decode(tok)])+" "+str(prob)+"%"
    score[0][torch.argmax(score[0])]-=100
    node+=1
    if tok==token_num:
        parent_next=node
    tree.create_node(string, node ,parent=parent)
    
    if parent==parent_next:
        node+=1
        parent_next=node
        tree.create_node(str(tokenizer.batch_decode(sequences[0:1,i+14])), node ,parent=parent)
        
    parent=parent_next
    i+=1

tree.show()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


sequences= <class 'torch.Tensor'> 30
['It is important for all countries to try harder to reduce carbon emissions because it is the only way to reduce global warming," he said.\n\n"']
[' because']
├── [' it'] 16.0%
│   ├── [' can'] 4.0%
│   ├── [' is'] 33.0%
│   │   ├── [' a'] 15.0%
│   │   ├── [' not'] 5.0%
│   │   └── [' the'] 11.0%
│   │       ├── [' key'] 7.0%
│   │       ├── [' major'] 7.0%
│   │       ├── [' only']
│   │       │   ├── [' option'] 2.0%
│   │       │   ├── [' thing'] 2.0%
│   │       │   └── [' way'] 70.0%
│   │       │       ├── [' of'] 4.0%
│   │       │       ├── [' to'] 73.0%
│   │       │       │   ├── [' achieve'] 6.0%
│   │       │       │   ├── [' prevent'] 5.0%
│   │       │       │   └── [' reduce'] 17.0%
│   │       │       │       ├── [' emissions'] 11.0%
│   │       │       │       ├── [' global'] 9.0%
│   │       │       │       │   ├── [' amount'] 5.0%
│   │       │       │       │   ├── [' global'] 7.0%
│   │       │       │       │   ├── [' warming

# P4

In [4]:
from openai import OpenAI

client = OpenAI(api_key ="")

sys_prom="You are a kind therapist, skilled in explaining cold matter in a warm way."
usr_prom="Turn the sentence \"You dislike the fact that your health is slowly deteriorating after each cigarette.\" into one that's softened, and non-expert so that it doesn't sound as authoritative nor certain. Generate 3 samples"

completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content":sys_prom },
        {"role": "user", "content":usr_prom }
    ],
    max_tokens=100
)
print(completion.choices[0].message.content)

It seems like you might be feeling a little concerned that smoking could be gradually affecting your wellbeing.


1. It seems like you're not fully enjoying the way your wellbeing gently changes with every cigarette.

2. It might be that you're noticing a shift in your overall wellness, seemingly connected with each smoke?

3. It seems like you might be feeling a little concerned that smoking could be gradually affecting your wellbeing.

In [3]:
print(completion.choices[0].message)

ChatCompletionMessage(content="1. It seems like you're not fully enjoying the way your wellbeing gently changes with every cigarette.\n\n2. It might be that you're noticing a shift in your overall wellness, seemingly connected with each smoke?\n\n3. You're possibly feeling a bit", role='assistant', function_call=None, tool_calls=None)


In [5]:
sys_prom="You are a kind therapist, skilled in explaining cold matter in a warm way."
usr_prom="Turn the sentence \"You were in a lot of pain.\" into one that's softened, and non-expert so that it doesn't sound as authoritative nor certain. Generate 1 samples"

completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content":sys_prom },
        {"role": "user", "content":usr_prom }
    ],
    max_tokens=100
)
print(completion.choices[0].message.content)

It seemed like you were going through a tough time.


In [25]:
f = open("DirectStatements.csv", "r",encoding='utf-8-sig')
sentences=[]
for i in f.readlines():
    sentences.append(i.strip())
   
for i in range(2,7,1):
    usr_prom="Turn the sentence \""+sentences[i]+"\" into one that's softened, and non-expert so that it doesn't sound as authoritative nor certain. Generate 1 samples"
    #print(usr_prom)
    completion = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content":sys_prom },
            {"role": "user", "content":usr_prom }
        ],
        max_tokens=100
    )
    print(completion.choices[0].message.content)

It seems like you might find some comfort in the soothing sensation that nicotine seems to provide you.
It seems like you might be reaching for the amphetamines a bit more than is usually suggested.
It seems like the scent of cigarette smoke might not be your favorite.
So, you've come today to share a bit about your experiences with gambling, haven't you?
You're beginning to see that by tweaking a few things, there could be some pretty significant shifts in your everyday world, and that's got you feeling a tad bit excited about all the possibilities, doesn't it?


In [30]:
from openai import OpenAI

client = OpenAI(api_key ="")

sys_prom="You are a kind therapist, skilled in explaining cold matter in a warm way."

f = open("DirectStatements.csv", "r",encoding='utf-8-sig')
fo=open("A4_4_4.csv", "w")

sentences=[]
for i in f.readlines():
    sentences.append(i.strip())

for i in range(7,9,1):
    usr_prom="\"Turn the sentence \"\""+sentences[i]+"\"\" into one that's softened, and non-expert so that it doesn't sound as authoritative nor certain. Generate 1 samples\""
    fo.write(sentences[i])
    fo.write(",\"")
    fo.write(sentences[i])
    fo.write("\",\n")
    
f.close()
fo.close()

# P5

In [33]:
sys_prom="You are a kind therapist, skilled in explaining cold matter in a warm way."
usr_prom=("You will be given a fen sentences and classify them based on softness. Here softness refers to whether a sentence is authoritative and direct, or is expressed as a suggestion or question. If a sentence is hard, print 0, and if it's soft, print 1. Here are the sentences: You dislike that others know that you smoke.\nYou want to stop smoking cigars altogether.\nYou seem low-energy.\nIt seems like you might be feeling a little concerned that smoking could be gradually affecting your wellbeing.\nIt seemed like you were going through a tough time.\nIt seems like you might find some comfort in the soothing sensation that nicotine seems to provide you.")

completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content":sys_prom },
        {"role": "user", "content":usr_prom }
    ],
    max_tokens=200
)
print(completion.choices[0].message.content)

0
0
0
1
1
1


In [55]:
sys_prom="You are a kind therapist, skilled in explaining cold matter in a warm way."
usr_prom="You will be given a fen sentences and classify them based on softness. Here softness refers to whether a sentence is authoritative, direct, stating, or is expressed as a suggestion or question. If a sentence is hard, print 0, and if it's soft, print 1. Here are the sentences:\n"


f = open("A4_5_1.csv", "r",encoding='utf-8-sig')

sentences=[]
for i in f.readlines():
    usr_prom=usr_prom+i.replace("0","").replace("1","")+"\n"

completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content":sys_prom },
        {"role": "user", "content":usr_prom }
    ],
    max_tokens=200
)
print(completion.choices[0].message.content)


KeyboardInterrupt



# P6

In [60]:
sys_prom="You are a kind therapist, skilled in explaining cold matter in a warm way."
usr_prom=("You will be given a few sentences and classify them based on softness. Here softness refers to whether a sentence is authoritative and direct, stating some facts in an unquestionable manner; or is expressed as a gentle suggestion for possibility or a question where the person speaking sounds as if they do not know the other party better than the other party themselves. Also keep in mind that just because a sentence is a question etc. doesn't mean it's soft. The subtlety of the question on sensitive topics e.g. wellbeing, social relations, self-confidence etc. must also be taken into consideration. For each sentence, explain why is it a soft/hard sentence instead of the other. If a sentence is hard, print 0, and if it's soft, print 1. Here are the sentences:\n")

f = open("A4_5_1.csv", "r",encoding='utf-8-sig')

sentences=[]
for i in f.readlines():
    usr_prom=usr_prom+i.replace("0","").replace("1","")+"\n"

completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content":sys_prom },
        {"role": "user", "content":usr_prom }
    ],
)
print(completion.choices[0].message.content)

1. 0 - This sentence is direct and specific about the person's health issue due to smoking, leaving little room for interpretation or exploration.
2. 0 - This statement asserts a fact about the person's past pain without any room for exploration or interpretation.
3. 0 - This statement asserts a fact about a person's enjoyment without room for exploration or nuanced interpretation.
4. 0 - This statement is making a direct judgement about a person's drug use without room for interpretation or their own evaluation.
5. 0 - This statement directly states the person's dislike, leaving no room for their own interpretation or exploration.
6. 1 - This sentence is soft as it's a question that softly inquires about the person's reason for the visit, leaving their response open-ended.
7. 0 - This sentence is hard because it assumes very specific feelings and expectations about the person's life changes.
8. 0 - This statement directly asserts a person's concern without any room for interpretation 